# Train on accident and person data


In [1]:
%reset 
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import xgboost as xgb
from numpy import nan as NA
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score as auc

Once deleted, variables cannot be recovered. Proceed (y/[n])? 
Nothing done.


# Load data and make labels consistent

In [2]:
print("Load labels")
label_file = pd.read_csv("./train/labels_ext.csv", index_col=0)

Load labels


In [3]:
Xtrain = pd.read_csv("./train/joint_accident_person_train_ext.csv", index_col=0)
n_train, n_dim = Xtrain.shape
print([n_train, n_dim])

[708569, 260]


In [ ]:
temp = pd.merge(Xtrain, label_file, on=['ID'], how='inner')

In [ ]:
y = temp['DRUNK_DR'].apply(lambda x: x*1 ).values

In [ ]:
%xdel temp

In [ ]:
indices = np.unique(Xtrain['ID'].values)
n_valid = int(len(indices)*0.3)
perm_indices = np.random.permutation(indices)
train_indices = perm_indices[:n_valid]
valid_indices = perm_indices[n_valid:]

# We will now group each accident by ID.
grouped = Xtrain.groupby('ID')

In [ ]:
tr_ids = {key : grouped.indices[key] for key in train_indices}
te_ids = {key : grouped.indices[key] for key in valid_indices}
%xdel grouped

In [ ]:
# Finally, we turn the dictionaries we created above into lists of row numbers.
# We can then use these row numbers to build our train/test split.
train_ids = []
for key in tr_ids:
    for ind in tr_ids[key].tolist():
        train_ids.append(ind)

valid_ids = []
for key in te_ids:
    for ind in te_ids[key].tolist():
        valid_ids.append(ind)    

In [ ]:
Xtrain.drop('ID', axis= 1, inplace=True)
print("Splitting data")
%xdel label_file
%xdel tr_ids
%xdel te_ids

In [ ]:
Xtrain_c = Xtrain.ix[train_ids].to_sparse()
Xeval = Xtrain.ix[valid_ids].to_sparse()
Ytrain_c = y[train_ids].to_sparse()
Yeval = y[valid_ids].to_sparse()

In [ ]:
%xdel Xtrain

%xdel valid_ids
%xdel train_ids

In [ ]:
%whos

In [ ]:
print("Load data into sparse matrix")
dtrain = xgb.DMatrix(data=Xtrain_c, missing = NA, label= Ytrain_c)
%xdel Xtrain_c

In [ ]:
deval  = xgb.DMatrix(data=Xeval, missing = NA, label = Yeval)
%xdel Xeval
print("Specifying the parameters ... ")

In [ ]:
%whos

In [ ]:
param = {'max_depth': 12,
         'eta': 0.02,
         'subsample': 0.7,
         'colsample_bytree': 0.8,
         'silent': 0,
         'eval_metric': 'auc',
         'alpha': 0,
         'lambda': 1,
         'nthread': 8,
         'objective': 'binary:logistic'}

In [ ]:
watchlist = [(deval, 'eval'), (dtrain, 'train')]
num_round = 520
print("Training ... ")

In [ ]:
bst = xgb.train(param, dtrain, num_round, watchlist)

In [ ]:
print("Saving the model")
bst.save_model('./models/xgb_acc_per.model')
bst.dump_model('./models/xgb_raw_acc_per.txt')

dtrain.save_binary('./models/binary/dtrain.buffer')
deval.save_binary('./models/binary/deval.buffer')

%xdel dtrain
%xdel deval

In [ ]:
%whos

In [ ]:
Xtest = pd.read_csv("./test/joint_accident_person_test_ext.csv", index_col=0)
ID= Xtest['ID'].astype(np.int64)
Xtest.drop('ID', axis= 1, inplace=True)
print("Load test samples")

In [ ]:
uniqueID = np.unique(ID)

In [ ]:
uniqueID

In [ ]:
dtest= xgb.DMatrix(data=Xtest.values, missing = NA)

In [ ]:
print("Prediction")
preds = bst.predict(dtest)

In [ ]:
dtest.save_binary('./models/binary/dtest.buffer')


In [ ]:
print("submit")
predict_df = pd.DataFrame(data={'ID': ID.values, 'prob': preds})
predict_df

In [ ]:
#predict_df.groupby('ID').mean()

In [ ]:
grouped_predict = predict_df.groupby('ID')
prediction = grouped_predict.max()

In [ ]:
prediction.index

In [ ]:
submit = pd.DataFrame(data={'ID': prediction.index, 'DRUNK_DR': prediction['prob'].values})
submit.to_csv('fars_submit.csv', index = False)
submit